In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
# contracts = populate_contracts(tickers)

In [15]:
from stockMarket.core import Technicals

for ticker in tickers:
    n_bars = 200
    contract = Technicals(ticker=ticker)
    contract.init_pricing_data(interval="weekly", n_bars=n_bars)
    for time in range(20, 52):
        time = -time
        
        if len(contract.pricing_data) < abs(time):
            continue
        
        last_candle = contract.pricing_data.iloc[time]

        candle_range = last_candle.high - last_candle.low
        fib_30 = last_candle.low + candle_range * 0.3
        fib_70 = last_candle.high - candle_range * 0.3
        
        ema_8 = contract.calc_ema(8)
        ema_20 = contract.calc_ema(20)
        ema_50 = contract.calc_ema(50)
        
        candle_classification = None
        if last_candle.open < fib_30 and last_candle.close < fib_30:
            candle_classification = "bearish"
        elif last_candle.open > fib_70 and last_candle.close > fib_70:
            candle_classification = "bullish"
            
        if candle_classification:
            print(f"{ticker} is showing a {candle_classification} candle on {last_candle.name}")
            
        ema_trend = False
        if candle_classification == "bearisch":
            ema_trend = ema_8[time] < ema_20[time] < ema_50[time]
            ema_trend &= last_candle.high >= ema_8[time]
        elif candle_classification == "bullish":
            ema_trend = ema_8[time] > ema_20[time] > ema_50[time]
            ema_trend &= last_candle.low <= ema_8[time]
        
        if ema_trend:
            print(f"    {ticker} {candle_classification} buy signal")

FICO is showing a bearish candle on 2023-09-18 15:30:00
BLDR is showing a bearish candle on 2023-10-09 15:30:00
BLDR is showing a bearish candle on 2023-07-17 15:30:00
BLDR is showing a bearish candle on 2023-04-17 15:30:00
AVGO is showing a bearish candle on 2023-08-21 15:30:00
AVGO is showing a bullish candle on 2023-05-08 15:30:00
    AVGO bullish buy signal
AVGO is showing a bearish candle on 2023-04-10 15:30:00
AVGO is showing a bullish candle on 2023-03-27 15:30:00
CVS is showing a bearish candle on 2023-07-03 15:30:00
BK is showing a bearish candle on 2023-10-23 15:30:00
BK is showing a bullish candle on 2023-10-02 15:30:00
BK is showing a bearish candle on 2023-07-24 15:30:00
BXP is showing a bullish candle on 2023-09-11 15:30:00
FAST is showing a bearish candle on 2023-09-25 15:30:00
FAST is showing a bullish candle on 2023-05-30 15:30:00
    FAST bullish buy signal
FAST is showing a bullish candle on 2023-05-22 15:30:00
    FAST bullish buy signal
CHRW is showing a bearish ca

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


TypeError: object of type 'NoneType' has no len()